In [1]:
import tweepy
import math
from datetime import datetime, timezone
import time

# Twitter API dati
bearer_token = "AAAAAAAAAAAAAAAAAAAAAKTn1wEAAAAAoA0PDvIRTBKAj4DEQc44vuUb0lY%3DCRSn3gqw7WWsTbAj769hbDBxB9lVeHrz405FQAawTpCQNeFiba"

client = tweepy.Client(bearer_token=bearer_token, wait_on_rate_limit=True)

In [2]:
def analyse_account(user_data):
    created_at = user_data.created_at.replace(tzinfo=timezone.utc)
    now = datetime.now(timezone.utc)
    account_age_days = (now - created_at).days

    weight = 0
    if account_age_days < 30:
        weight += (30 - account_age_days)  # jo jaunāks, jo aizdomīgāks
    elif user_data.public_metrics["followers_count"] < 10:
        weight += 10

    if not user_data.verified:
        weight += 50
    else:
        weight -= 10

    return weight

In [3]:
def analyse_tweets_similarity(tweets):
    unique_texts = set()
    for tweet in tweets:
        unique_texts.add(tweet.text.strip())

    total = len(tweets)
    if total == 0:
        return 0

    return 100 * (total - len(unique_texts)) / total

In [4]:
def tweet_time_intervals(tweets):
    if len(tweets) < 2:
        return 0

    timestamps = [tweet.created_at.timestamp() for tweet in tweets]
    timestamps.sort()

    intervals = [timestamps[i+1] - timestamps[i] for i in range(len(timestamps)-1)]
    mean_interval = sum(intervals) / len(intervals)
    variance = sum((x - mean_interval)**2 for x in intervals) / len(intervals)
    std_dev = math.sqrt(variance)

    weight = 0
    if std_dev < 300:  # mazāka par 5 minūtēm
        weight += 20
    if std_dev > 0:
        weight += 400 / std_dev
    return weight

In [5]:
def bot_score(username, max_tweets=10):
    try:
        user = client.get_user(username=username, user_fields=["created_at", "public_metrics", "verified"]).data
        tweets = client.get_users_tweets(user.id, max_results=min(max_tweets, 100), tweet_fields=["created_at"]).data or []

        score = 0
        score += analyse_account(user)
        score += analyse_tweets_similarity(tweets)
        score += tweet_time_intervals(tweets)
        return score
    except Exception as e:
        print(f"Error analyzing @{username}: {e}")
        return None

In [6]:
def is_bot(score, threshold=50):
    if score is None:
        return "Unknown"
    return score >= threshold

In [7]:
username = "khazi_messi"
score = bot_score(username)
print(f"Bot score for @{username}: {score}")
print("Classification:", "Bot" if is_bot(score) else "Human")

Bot score for @khazi_messi: 60.0019791004183
Classification: Bot
